In [ ]:
# acc, wrong avg tokens, right avg tokens, avg flops, corrct_tokens_ratio

In [1]:
from transformers import AutoConfig
configs = {
    '1b': AutoConfig.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct'),
    '7b': AutoConfig.from_pretrained('Qwen/Qwen2.5-7B-Instruct'),
    '14b': AutoConfig.from_pretrained('Qwen/Qwen2.5-14B-Instruct'),
    '32b': AutoConfig.from_pretrained('Qwen/Qwen2.5-32B-Instruct')
}

def many_decode_flops(p_l, s, h, h_, n):
    return 8*s*(h^2) + 16*s*h + 4*s*p_l*h + 4*s*p_l*n + 6*s*h*h_ + 2*s*h_

def prefill_flop(s, h, h_, n):
    return 8*s*(h^2) + 16*s*h + 4*(s^2)*h + 4*(s^2)*n + 6*s*h*h_ + 2*s*h_

def decode_flop(s, h, h_, n):
    return 8*(h^2) + 16*h + 4*s*h + 4*s*n + 6*h*h_ + 2*h_

def cum_decode_flop(p_l, l, h, h_, n):
    flops = 0
    for i in range(l):
        flops = flops + decode_flop(p_l+i, h, h_, n)
    return flops

def get_flops(name, help_model, token_usages):
    correct_num = 0
    if help_model not in configs.keys():
        config = configs[name]
        info= config.hidden_size, config.intermediate_size, config.num_attention_heads
        flops = prefill_flop(token_usages['prompt_tokens'], *info) + cum_decode_flop(token_usages['prompt_tokens'], token_usages['completion_tokens'], *info)
    else:
        spe_config = configs[name]
        tgt_config = configs[help_model]
        spe_info = spe_config.hidden_size, spe_config.intermediate_size, spe_config.num_attention_heads
        tgt_info = tgt_config.hidden_size, tgt_config.intermediate_size, tgt_config.num_attention_heads
        correct_tokens = token_usages['correct_tokens']
        completion_tokens = token_usages['completion_tokens']
        prompt_tokens = token_usages['prompt_tokens']
        decode_spe_flops, decode_tgt_flops = 0, 0
        last_spe_decode_pos = prompt_tokens
        token_num = 0
        decode_spe_flops = decode_spe_flops+prefill_flop(prompt_tokens, *spe_info)
        decode_tgt_flops = decode_tgt_flops+prefill_flop(prompt_tokens, *tgt_info)
        for cor_t in correct_tokens:
            pos, token_num = cor_t['pos']+prompt_tokens-token_num, cor_t['token_num']
            decode_spe_flops = decode_spe_flops+cum_decode_flop(last_spe_decode_pos, pos-last_spe_decode_pos, *spe_info)
            decode_tgt_flops = decode_tgt_flops+many_decode_flops(last_spe_decode_pos, pos-last_spe_decode_pos, *tgt_info)
            decode_tgt_flops = decode_tgt_flops+cum_decode_flop(pos, token_num, *tgt_info)
            decode_spe_flops = decode_spe_flops+many_decode_flops(pos, token_num, *spe_info)
            correct_num = correct_num + token_num
            last_spe_decode_pos = pos
        decode_spe_flops = decode_spe_flops+cum_decode_flop(last_spe_decode_pos+token_num, completion_tokens+prompt_tokens-last_spe_decode_pos-token_num, *spe_info)
        # print(token_usages)
        flops = decode_spe_flops+decode_tgt_flops
    return flops, correct_num


/home/wxy320/miniconda3/envs/multio1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
print(name, model)
t=0.6
spe_config = configs[name]
tgt_config = configs[model]
spe_info = spe_config.hidden_size, spe_config.intermediate_size, spe_config.num_attention_heads
tgt_info = tgt_config.hidden_size, tgt_config.intermediate_size, tgt_config.num_attention_heads
for k in data.keys():
    print(data[k].keys())
    token_usages = data[k]['token_usages'][str(t)][0]
    correct_tokens = token_usages['correct_tokens']
    completion_tokens = token_usages['completion_tokens']
    prompt_tokens = token_usages['prompt_tokens']
    decode_spe_flops, decode_tgt_flops = 0, 0
    last_spe_decode_pos = completion_tokens
    correct_num, token_num = 0, 0
    for cor_t in correct_tokens:
        pos, token_num = cor_t['pos']+prompt_tokens, cor_t['token_num']
        decode_spe_flops = decode_spe_flops+cum_decode_flop(last_spe_decode_pos, pos-last_spe_decode_pos, *spe_info)
        decode_tgt_flops = decode_tgt_flops+cum_decode_flop(pos, token_num, *tgt_info)
        correct_num = correct_num + token_num
        last_spe_decode_pos = pos
    decode_spe_flops = decode_spe_flops+prefill_flop(correct_num+prompt_tokens, *spe_info)
    decode_tgt_flops = decode_tgt_flops+prefill_flop(completion_tokens-correct_num, *tgt_info)
    print(token_usages)
    flops = decode_spe_flops+decode_tgt_flops
    print(flops)
    break

7b 32b
dict_keys(['problem', 'solution', 'answer', 'subject', 'level', 'unique_id', 'responses', 'token_usages', 'prompt', 'input_conversation'])
{'completion_tokens': 1534, 'prompt_tokens': 67, 'correct_tokens': [{'pos': 629, 'token_num': 20, 'traget': 'Wait, let me double-check that. 3 is 9/3, adding 5/', 'speculative': 'Wait, let me check that again. 3 is indeed 9/3, so 9'}, {'pos': 649, 'token_num': 125, 'traget': "3 gives 14/3. Yeah, that's correct. So, the total is 14/3. Hmm, is that the simplest form? 14 and 3 don't have any common factors besides 1, so yes, 14/3 is the simplified form.\n\nLet me just verify each step to make sure I didn't make a mistake. \n\nFirst, f(-2): 3*(-2) is -6, minus 2 is -8. Denominator: -2 - 2 is -4. So, -8/-4 is 2. That seems"}, {'pos': 924, 'token_num': 20, 'traget': 'Wait, just to make sure, maybe I can compute the sum another way. Let me add all', 'speculative': 'Wait, just thinking about the function, is there a simpler way to approach this? Mayb

In [2]:
import os
import json

def get_info(data, name, help_model, t=0.6):
    correct_num, correct_tokens, worng_tokens, all_time_spend = 0, 0, 0, 0
    fix_tokens, right_flops, wrong_flops, all_flops = 0, 0, 0, 0
    for k in data.keys():
        # print(data[k].keys())
        token_usages = data[k]['token_usages'][str(t)][0]
        # time_spend = token_usages['time_spend']
        tokens_num = token_usages['completion_tokens']
        flops, fix_token = get_flops(name, help_model, token_usages)
        if data[k]['responses'][str(t)][0]['correctness']: 
            correct_num = correct_num+1
            correct_tokens = correct_tokens+tokens_num
            right_flops = right_flops+flops
        else:
            worng_tokens = worng_tokens+tokens_num
            wrong_flops = wrong_flops+flops
        all_flops = all_flops+flops
        fix_tokens = fix_tokens+fix_token
        #  all_time_spend = all_time_spend + time_spend
    all_num = len(data.keys())
    r = 10000000000
    return {
        'all_num':all_num,
        'acc':round(correct_num/all_num, 4),
        'correct_avg_tokens':round(correct_tokens/correct_num, 2),
        'worng_avg_tokens':round(worng_tokens/(all_num-correct_num), 2),
        # 'right_flops': round(right_flops/(correct_num*r), 2),
        # 'wrong_flops': round(wrong_flops/((all_num-correct_num)*r), 2),
        'avg_flops': round(all_flops/(all_num*r), 2),
        #  'avg_time_spend': round((correct_tokens+worng_tokens)/all_time_spend, 2),
        'fix_tokens': round(fix_tokens/(correct_tokens+worng_tokens),2)
    }

In [3]:
from collections import defaultdict
results = defaultdict(dict)

folder_path = "/home/wxy320/ondemand/program/speculative_thinking/analysis/1/normal"  # 替换为你的文件夹路径
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
for file in json_files:
    name = file.split('_')[0].split('-')[-1].lower()
    if name == '1.5b': name = '1b'
    dataset = file.split('_')[1]
    file_path = os.path.join(folder_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        result = get_info(data, name, None)
    results[(name,dataset)] = {
        'normal': result
    }

In [4]:
folder_path = "/home/wxy320/ondemand/program/speculative_thinking/analysis/1/speculative"  # 替换为你的文件夹路径
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

for file in json_files:
    name, model, dataset = file.split('_')[0], file.split('_')[1].split('.')[0], file.split('_')[2]
    file_path = os.path.join(folder_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        result = get_info(data, name, model)
        # print(result)
    results[(name,dataset)][model] = result

In [5]:
final_results = defaultdict(dict)
for key, r in results.items():
    # print(key)
    final_results[key[1]][key[0]]=r
for d, r in final_results.items():
    print(d)
    print('------------------------')
    for k, r_ in r.items():
        print(k)
        for k_, r_1 in r_.items():
            print(k_, r_1)
    print('------------------------')

aime22
------------------------
1b
normal {'all_num': 30, 'acc': 0.3, 'correct_avg_tokens': 9296.56, 'worng_avg_tokens': 21558.86, 'avg_flops': 279.74, 'fix_tokens': 0.0}
14b {'all_num': 30, 'acc': 0.3333, 'correct_avg_tokens': 7319.2, 'worng_avg_tokens': 22248.6, 'avg_flops': 1643.81, 'fix_tokens': 0.17}
32b {'all_num': 30, 'acc': 0.2667, 'correct_avg_tokens': 7282.12, 'worng_avg_tokens': 18987.59, 'avg_flops': 2153.17, 'fix_tokens': 0.17}
7b
normal {'all_num': 30, 'acc': 0.5, 'correct_avg_tokens': 9178.8, 'worng_avg_tokens': 16906.67, 'avg_flops': 720.87, 'fix_tokens': 0.0}
14b {'all_num': 30, 'acc': 0.4667, 'correct_avg_tokens': 9469.93, 'worng_avg_tokens': 22208.62, 'avg_flops': 2302.67, 'fix_tokens': 0.18}
32b {'all_num': 30, 'acc': 0.5667, 'correct_avg_tokens': 7540.53, 'worng_avg_tokens': 22824.08, 'avg_flops': 2645.4, 'fix_tokens': 0.18}
14b
normal {'all_num': 30, 'acc': 0.6, 'correct_avg_tokens': 7878.28, 'worng_avg_tokens': 22725.0, 'avg_flops': 873.5, 'fix_tokens': 0.0}
32b
